# Analyzing graph distance and coverage

Figure 2D and "Physical" baseline in Table 1.

In [1]:
import os
import json
import pickle

from tqdm import tqdm

import numpy as np
import pandas as pd
import networkx as nx

from sklearn.metrics import roc_auc_score

In [2]:
def get_pairs(fp):
    df = pd.read_csv(fp)
    df = df[df["split"] == "test"]
    pert = df["pert"]
    gene = df["gene"]
    label = df["label"]
    pairs = zip(pert, gene)
    return list(pairs), label.tolist()

In [3]:
cell_lines = ["k562", "rpe1", "hepg2", "jurkat"]
all_pairs = {}
all_labels = {}
for cl in cell_lines:
    fp = f"../perturbqa/datasets/{cl}-de.csv"
    pairs, labels = get_pairs(fp)
    all_pairs[cl] = pairs
    all_labels[cl] = labels

## KG connectivity

In [4]:
def load_graph(fp):
    with open(fp) as f:
        g = json.load(f)
    return g

In [5]:
pathway_graph = nx.Graph()
gene_graph = nx.Graph()

bad = ["GO:0005515", "GO:0003674"]  # "protein binding" ...

for name in ["string", "corum", "go", "reactome"]:
    fp = f"../perturbqa/datasets/kg/{name}.json"
    gene2rel = load_graph(fp)
    if len(gene2rel) <= 3:
        graph = pathway_graph
        gene2rel = gene2rel[0]
    else:
        graph = gene_graph
    # add to big graph
    for gene, edges in gene2rel.items():
        for node2, _ in edges:
            if node2 in bad:
                continue
            graph.add_edge(gene, node2)

go2go = load_graph("../perturbqa/datasets/kg/go.json")[2]
for node1, edges in go2go.items():
    if node1 in bad:
        continue
    for node2, _ in edges:
        if node2 in bad:
            continue
        pathway_graph.add_edge(node1, node2)

print(pathway_graph, gene_graph)

Graph with 80043 nodes and 495273 edges Graph with 18479 nodes and 752612 edges


In [6]:
to_delete = []
large = []
for node in pathway_graph:
    if nx.degree(pathway_graph, node) >= 1000:
        to_delete.append(node)
    elif nx.degree(pathway_graph, node) >= 500:
        large.append((node, nx.degree(pathway_graph, node)))

for node in to_delete:
    pathway_graph.remove_node(node)

In [7]:
with open("../perturbqa/datasets/kg/go_dict.json") as f:
    go_dict = json.load(f)

In [8]:
[(go_dict[x[0]], x[1]) for x in large]

[('innate immune response', 519),
 ('intracellular membrane-bounded organelle', 901),
 ('calcium ion binding', 720),
 ('centrosome', 647),
 ('G protein-coupled receptor activity', 742),
 ('G protein-coupled receptor signaling pathway', 961),
 ('synapse', 500),
 ('DNA binding', 983),
 ('positive regulation of cell population proliferation', 510),
 ('negative regulation of DNA-templated transcription', 572),
 ('protein homodimerization activity', 726),
 ('Golgi membrane', 691),
 ('proteolysis', 513),
 ('cell differentiation', 670),
 ('regulation of DNA-templated transcription', 569),
 ('biological_process', 593),
 ('nucleolus', 946),
 ('zinc ion binding', 859),
 ('negative regulation of transcription by RNA polymerase II', 927),
 ('sequence-specific double-stranded DNA binding', 568),
 ('perinuclear region of cytoplasm', 723),
 ('positive regulation of DNA-templated transcription', 705),
 ('apoptotic process', 591),
 ('protein-containing complex', 914),
 ('cell surface', 625)]

Compute path lengths

In [9]:
all_lengths_path = {}
all_lengths_gene = {}
for cl, pairs in all_pairs.items():
    all_lengths_path[cl] = []
    all_lengths_gene[cl] = []
    # pathway version
    for g1, g2 in tqdm(pairs):
        if g1 not in pathway_graph or g2 not in pathway_graph:
            all_lengths_path[cl].append(-1)
            continue
        try:
            length = nx.shortest_path_length(pathway_graph, g1, g2)
            all_lengths_path[cl].append(length)
        except nx.NetworkXNoPath:
            all_lengths_path[cl].append(-1)

    # gene to gene version
    for g1, g2 in tqdm(pairs):
        if g1 not in gene_graph or g2 not in gene_graph:
            all_lengths_gene[cl].append(-1)
            continue
        try:
            length = nx.shortest_path_length(gene_graph, g1, g2)
            all_lengths_gene[cl].append(length)
        except nx.NetworkXNoPath:
            all_lengths_gene[cl].append(-1)


100%|████████████████████████████████████| 29138/29138 [00:01<00:00, 24208.90it/s]


## Observations

Observation: positive pairs significantly *more* likely to interact physically

In [10]:
for cl, lengths in all_lengths_gene.items():
    pos_1 = 0
    neg_1 = 0
    assert len(all_labels[cl]) == len(lengths)
    for lbl, length in zip(all_labels[cl], lengths):
        if length == 1 and lbl == 1:
            pos_1 += 1
        elif length == 1 and lbl == 0:
            neg_1 += 1
    total = len(lengths)
    print(cl, pos_1, neg_1, total)
    total_pos = all_labels[cl].count(1)
    total_neg = all_labels[cl].count(0)

    # p-value
    successes = pos_1, neg_1
    n_obs = total_pos, total_neg
    print(f"{pos_1/total_pos:.3f} {neg_1/total_neg:.3f}")
    print(f"{(pos_1 + neg_1)/len(all_labels[cl]):.3f}\n")

k562 294 448 23212
0.094 0.022
0.032

rpe1 337 602 37942
0.063 0.018
0.025

hepg2 269 420 25749
0.075 0.019
0.027

jurkat 436 421 29138
0.106 0.017
0.029



Observation: positive pairs *not* more likely to interact via network / pathway

In [11]:
for cl, lengths in all_lengths_path.items():
    pos_2 = 0
    neg_2 = 0
    assert len(all_labels[cl]) == len(lengths)
    for lbl, length in zip(all_labels[cl], lengths):
        if length == 2 and lbl == 1:
            pos_2 += 1
        elif length == 2 and lbl == 0:
            neg_2 += 2
    total_pos = all_labels[cl].count(1)
    total_neg = all_labels[cl].count(0)

    # p-value
    successes = pos_2, neg_2
    n_obs = total_pos, total_neg

    print(cl, pos_2, neg_2, total)
    print(f"{pos_2/total_pos:.3f} {neg_2/total_neg:.3f}")
    print(f"{(pos_2 + neg_2)/len(all_labels[cl]):.3f}\n")

k562 669 4488 29138
0.214 0.223
0.222

rpe1 1097 6830 29138
0.204 0.210
0.209

hepg2 786 4888 29138
0.218 0.221
0.220

jurkat 1044 5014 29138
0.253 0.200
0.208



Observation: physical connection is minimally predictive of differential expression

In [12]:
for cl, lengths in all_lengths_gene.items():
    pred = []
    assert len(all_labels[cl]) == len(lengths)
    for lbl, length in zip(all_labels[cl], lengths):
        if length == 1:
            pred.append(1)
        else:
            pred.append(0)
    true = all_labels[cl]
    auc = roc_auc_score(true, pred)
    print(f"{cl}: {auc:.3f}")


k562: 0.536
rpe1: 0.522
hepg2: 0.528
jurkat: 0.544


Observation: size of neighborhoods grows exponentially as distance (1 vs. 2 hops depicted)

In [13]:
deg1 = []
for node in gene_graph:
    deg1.append(nx.degree(gene_graph, node))
np.quantile(deg1, [0, .25, .5, .75, 1])

array([1.00e+00, 1.50e+01, 4.00e+01, 9.30e+01, 2.94e+03])

In [14]:
deg2 = []
for node in tqdm(gene_graph):
    neighbors = nx.neighbors(gene_graph, node)
    neighbors_2hop = set()
    for node2 in neighbors:
        neighbors_2hop.update(nx.neighbors(gene_graph, node2))
    deg2.append(len(neighbors_2hop))
np.quantile(deg2, [0, .25, .5, .75, 1])

100%|██████████████████████████████████████| 18479/18479 [00:25<00:00, 730.75it/s]


array([1.000e+00, 2.040e+03, 4.456e+03, 7.584e+03, 1.726e+04])

In [15]:
np.quantile(deg2, [0, .25, .5, .75, 1])

array([1.000e+00, 2.040e+03, 4.456e+03, 7.584e+03, 1.726e+04])